In [3]:
import eikon as ek
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly.graph_objs as go
import plotly.subplots as subplots

from datetime import datetime
from plotly.offline import init_notebook_mode, iplot, plot

cf.go_offline()
init_notebook_mode(connected=True)

APP_KEY = 'YOUR_APP_KEY'
ek.set_app_key(APP_KEY)

direct_qm = ['AUD=', 'BTC=', 'BWP=', 'EUR=', 'FJD=', 'FKP=', 'GBP=', 'GIP=', 'IEP=', 'MTL=', 'NZD=', 'PGK=', 'SBD=', 'SHP=', 'TOP=', 'USD=', 'WST=', 'XAG=', 'XAU=','XPD=', 'XPT=']
z_score_const = 1.64485

def convert_to_usd(ric, rate, amt):
    if ric in direct_qm:
        return amt * rate
    else:
        if rate != 0:
            return amt / rate
        else:
            return 0

def get_annual_stdev(risk_array, ann_stdev):
    arr_A = np.array(risk_array)
    arr_B = np.array(ann_stdev)
    annual_stdev = list(np.multiply(arr_A, arr_B))
    return annual_stdev

def get_port_vol(corr_mtx, stdev):
    _v = pd.DataFrame(corr_mtx.dot(stdev)).transpose()
    port_vol = np.dot(np.array(_v), np.array(pd.DataFrame(stdev)))[0][0] ** 0.5
    return port_vol
        
def get_data(portfolio):
    instruments = list(portfolio.keys())
    df = ek.get_timeseries(instruments, 'CLOSE', interval='daily', count=252)
    df = df.fillna(method='ffill')
    positions = list(map(lambda p: portfolio[p][0], instruments))
    hedge_ratios = list(map(lambda p: portfolio[p][1], instruments))
    spot_fx = df.tail(1).values.tolist()[0]
    flow_usd = list(map(lambda r, f, a: convert_to_usd(r, f, a), instruments, spot_fx, positions))
    risk_amt_before_hedge = round(sum(flow_usd), 2)
    risk_pct_before_hedge = list(np.array(flow_usd) / risk_amt_before_hedge)
    hedge_usd = list(np.multiply(hedge_ratios, flow_usd))
    remaining_flow_usd = list(np.subtract(np.array(flow_usd), np.array(hedge_usd)))
    risk_amt_after_hedge = round(sum(remaining_flow_usd), 2)
    risk_pct_after_hedge = list(np.array(remaining_flow_usd) / risk_amt_after_hedge)

    #calculate log returns
    df_log = pd.DataFrame()
    df_log = np.log(df) - np.log(df.shift(1))
    df_log = df_log.dropna(axis=0)
    df_stats = df_log.describe()
    daily_std = list(map(lambda c: df_stats[c]['std'], list(df_stats.columns)))
    daily_avg = list(map(lambda c: df_stats[c]['mean'], list(df_stats.columns)))
    annualized_std = list(map(lambda x: x * (252 ** 0.5), daily_std))
    #calculate excess returns
    df_x_ret = df_log
    for n, c in enumerate(df_x_ret.columns):
        df_x_ret[c] -= daily_avg[n]

    ann_std_hedged = get_annual_stdev(risk_pct_after_hedge, annualized_std)
    ann_std_unhedged = get_annual_stdev(risk_pct_before_hedge, annualized_std)
    
    #create correlation matrix on excess returns & vols
    corr_mtx = df_x_ret.corr()
    
    vol_after_hedge = get_port_vol(corr_mtx, ann_std_hedged)
    vol_before_hedge = get_port_vol(corr_mtx, ann_std_unhedged)
    var_before_hedge = (vol_before_hedge * np.abs(risk_amt_before_hedge) * z_score_const).round(2)
    var_after_hedge = (vol_after_hedge * np.abs(risk_amt_after_hedge) * z_score_const).round(2)
    
    df_pct_risk = pd.DataFrame()
    df_pct_risk['% Risk after hedge'] = risk_pct_after_hedge
    df_pct_risk['% Risk before hedge'] = risk_pct_before_hedge
    df_pct_risk.index = instruments
    value_at_risk = pd.DataFrame()
    value_at_risk['Risk amount, USD'] = [risk_amt_before_hedge, risk_amt_after_hedge]
    value_at_risk['Volatility'] = [vol_before_hedge, vol_after_hedge]
    value_at_risk['VaR, USD'] = [var_before_hedge, var_after_hedge]
    value_at_risk.index = ['Before hedge', 'After hedge']
    
    #charting
    trace1 = go.Heatmap(x = df_pct_risk.index, y = df_pct_risk.columns, z = df_pct_risk.T.values * 100, colorscale='YlGnBu')
    trace2 = go.Bar(x = value_at_risk.index, y = value_at_risk['VaR, USD'].values, marker=dict(color='darkblue'))
    fig = subplots.make_subplots(shared_yaxes=False, shared_xaxes=False, rows=1, cols=2, 
                              subplot_titles = ('VaR chg, USD','Risk distribution chg, %'), print_grid = False)
    fig.append_trace(trace1, row= 1, col=2)
    fig.append_trace(trace2, row= 1, col=1)
    fig.layout.title = 'FX hedge impact analysis'
    fig.layout.xaxis=dict(domain = [0, 0.25])
    fig.layout.xaxis2=dict(domain = [0.45, 1])
    iplot(fig)    
    return

